In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
import cv2
import time
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import math



/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
seed = 4
#read in the files, if the images are smaller than [max_rows,max_cols] pad them, if greater we won't use them
numChannels = 1
max_rows = 200
max_cols = 200 #chosen by looking at a histogram of img sizes
learning_rate = 0.0001#10e-4 works
Epochs = 100
batch_size = 100
dropout = .5
test_size = .05
num_input = max_rows*max_cols*numChannels
num_classes = len(names)
numOut1 = 32
numOut2 = 64
numOut3 = 128
numOut4 = 256
numOutF = 1024
numOutF2 = 512

In [4]:
img_holder = np.zeros(max_rows*max_cols*numChannels)
img_file_holder = []
img_class_holder = []
#determine which files will be good to use and how many of them there are so we can preallocate memory l8er
for name in names:
    fdir = os.path.join(direc,name)
    for f in os.listdir(fdir):
        if f[-3:] == 'jpg':
            fimg = os.path.join(fdir,f)
            img_file_holder.append(fimg)
            img_class_holder.append(name)




In [ ]:
len(img_file_holder)

In [5]:
img_holder = np.zeros((len(img_file_holder),max_rows*max_cols*numChannels),dtype=np.float32) #preallocate memory to hold all img data
#squeeze the img into flat arrays and put in holder
for idx,fimg in enumerate(img_file_holder):
    img = cv2.imread(fimg)
    

    [rows,cols] = np.shape(img)[0:2]
    if rows < max_rows or cols < max_cols:
        #smaller
        resized = cv2.resize(img,(max_rows,max_cols),interpolation = cv2.INTER_CUBIC)
    else:
        #bigger
        resized = cv2.resize(img,(max_rows,max_cols),interpolation = cv2.INTER_AREA)
    
    resized = cv2.cvtColor(resized,cv2.COLOR_RGB2GRAY)

    padImg = np.zeros((max_rows,max_cols,3),dtype = np.int8)
    padR = max_rows-rows
    padC = max_cols-cols
    row0 = np.floor(padR/2).astype(int)
    col0 = np.floor(padC/2).astype(int)
    row1 = row0+rows
    col1 = col0+cols
    padImg[row0:row1,col0:col1,:] = img
    flatImg = np.reshape(resized,-1)
    flatImg = (flatImg - np.mean(flatImg))/np.std(flatImg)
    flatImg = (flatImg-np.min(flatImg))
    flatImg = flatImg/np.max(flatImg)
    img_holder[idx,:] = flatImg
#del img_file_holder

ValueError: could not broadcast input array from shape (263,320,3) into shape (32,60,3)

In [ ]:

le = LabelEncoder()
le.fit(img_class_holder)
Y = le.transform(img_class_holder)
Y = np_utils.to_categorical(Y)
(x_train,x_test,y_train,y_test) = train_test_split(img_holder,Y,test_size = test_size)
#del img_holder


In [ ]:

def luminancy(RGBimg):
    return .3*RGBimg[:,:,0]+.6*RGBimg[:,:,1]+.1*RGBimg[:,:,2]
img_holder = np.zeros((len(img_file_holder),max_rows*max_cols*numChannels),dtype=np.float32) #preallocate memory to hold all img data
#squeeze the img into flat arrays and put in holder
for idx,fimg in enumerate(img_file_holder):
    img = cv2.imread(fimg)
    [rows,cols] = np.shape(img)[0:2]
    img = luminancy(img)
    #img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    if rows < max_rows: 
        #smaller
        diff = max_rows-rows
        padd = np.zeros([diff,cols])
        img = np.concatenate((img,padd),0)
    else:
        #bigger
        diff = rows-max_rows
        if diff%2 == 0:
            #even diff
            img = img[int(diff/2):rows-int(diff/2),:]
        else:
            #odd
            img = img[math.ceil(diff/2):rows-math.ceil(diff/2)+1,:]
    [rows,cols] = np.shape(img)
    if cols < max_cols:
        diff = max_cols-cols
        padd = np.zeros([rows,diff])
        img = np.concatenate((img,padd),1)
    else:
        #bigger
        diff = int(cols-max_cols)
        if diff%2 == 0:
            #even diff
            img = img[:,int(diff/2):cols-int(diff/2)]
        else:
            #odd
            img = img[:,math.ceil(diff/2):cols-math.ceil(diff/2)+1]
    flatImg = np.reshape(img,-1)
    flatImg = (flatImg - np.mean(flatImg))/np.std(flatImg)
    flatImg = (flatImg-np.min(flatImg))
    flatImg = flatImg/np.max(flatImg)
    img_holder[idx,:] = flatImg
    

le = LabelEncoder()
le.fit(img_class_holder)
Y = le.transform(img_class_holder)
Y = np_utils.to_categorical(Y)
(x_train,x_test,y_train,y_test) = train_test_split(img_holder,Y,test_size = test_size,random_state=seed)
        
    




In [ ]:
imtest = img_holder[509]

imtest = (255*imtest).astype('int8')
imtest = np.reshape(imtest,[max_rows,max_cols])



